In [11]:
import os, json, time
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import time
import numpy as np
import os
import random 
import matplotlib.pyplot as plt
import math
from operator import itemgetter
from tqdm import tqdm
import math
import pickle

In [12]:
# Mappings of Class Names
objects_classes = {'Legend': 0, 'yAxisTitle': 1, 'ChartTitle':2, 'xAxisTitle': 3, 'LegendPreview':4, 'PlotArea':5,
'yAxisLabel':6, 'xAxisLabel':7, 'LegendLabel': 8, 'PieLabel':9, 'bar':10, 'pie':11, 'line': 12, 'pie_slice':13, 'dot_line':14 }

# Helper Functions

In [13]:
def preprocess_line_chart(image):

    bounding_boxes = []
    masks = []
    for model in image['models']:
        bboxes = model['bboxes']
        y_vals = model['y']
        min_x0 = math.inf
        min_y0 = math.inf
        max_x1 = -math.inf
        max_y1 = -math.inf
        top_points = []
        bottom_points = []
        for i, bbox in enumerate(bboxes):
            
            y0,x0,w,h = float(bbox["y"]), float(bbox["x"]), float(bbox["w"]), float(bbox["h"])
            y1, x1 = y0 + h, x0 + w
            min_x0 = min(min_x0, x0) 
            min_y0 = min(min_y0, y0) 
            max_x1 = max(max_x1, x1) 
            max_y1 = max(max_y1, y1)

        masks.append([min_x0, min_y0, max_x1, min_y0, max_x1, max_y1, min_x0, max_y1])
        bounding_boxes.append((objects_classes['line'], [min_x0, min_y0, max_x1, max_y1]))

    return bounding_boxes, masks

def preprocess_dot_line_chart(image):

    bounding_boxes = []
    masks = []
    for model in image['models']:
        bboxes = model['bboxes']
        y_vals = model['y']
        min_x0 = math.inf
        min_y0 = math.inf
        max_x1 = -math.inf
        max_y1 = -math.inf
        top_points = []
        bottom_points = []
        for i, bbox in enumerate(bboxes):
            
            y0,x0,w,h = float(bbox["y"]), float(bbox["x"]), float(bbox["w"]), float(bbox["h"])
            y1, x1 = y0 + h, x0 + w
            min_x0 = min(min_x0, x0) 
            min_y0 = min(min_y0, y0) 
            max_x1 = max(max_x1, x1) 
            max_y1 = max(max_y1, y1)

        masks.append([min_x0, min_y0, max_x1, min_y0, max_x1, max_y1, min_x0, max_y1])
        bounding_boxes.append((objects_classes['dot_line'], [min_x0, min_y0, max_x1, max_y1]))

    return bounding_boxes, masks

def preprocess_bar_chart(image):
    bounding_boxes = []
    masks = []
    for model in image['models']:
        bboxes = model['bboxes']
        for bbox in bboxes:
            y0,x0,w,h = float(bbox["y"]), float(bbox["x"]), float(bbox["w"]), float(bbox["h"])
            y1, x1 = y0 + h, x0 + w
            bounding_boxes.append((objects_classes['bar'], [x0, y0, x1, y1]))
            masks.append([x0, y0, x1, y0, x1, y1, x0, y1])
    return bounding_boxes, masks


def preprocess_pie_chart(image):
    
    bounding_boxes = []
    starts_ends = []
    masks = []
    min_x0 = math.inf
    min_y0 = math.inf
    max_x1 = -math.inf
    max_y1 = -math.inf
    # Marks
    for model in image['models']:
        bbox = model['bbox']
        y0,x0,w,h = float(bbox["y"]), float(bbox["x"]), float(bbox["w"]), float(bbox["h"])
        y1, x1 = y0 + h, x0 + w
        min_x0 = min(min_x0, x0) 
        min_y0 = min(min_y0, y0) 
        max_x1 = max(max_x1, x1) 
        max_y1 = max(max_y1, y1)
        bounding_boxes.append((objects_classes['pie_slice'], [x0, y0, x1, y1]))
        starts_ends.append([model['start'], model['end']])
    

    center_x = (min_x0 + max_x1)/2
    center_y = (min_y0 + max_y1)/2

    top_x = center_x
    top_y = min_y0

    radius = math.hypot(top_x - center_x, top_y - center_y)
    for item in starts_ends:
        start, end = item
        from math import cos, sin, pi
        mask = []
        for angle in np.arange(float(start),float(end),0.017):#range(start, end, 0.017):
            x = float(center_x) + float(radius * cos(angle))
            y = float(center_y) - float(radius * sin(angle))
            mask.extend([x,y])

        #x1 = center_x + (radius * cos(end))
        #y1 = center_y + (radius * sin(end))
        #mask.extend([x1, y1])
        mask.extend([center_x, center_y])
        masks.append(mask)
          
    
    bounding_boxes.append((objects_classes['pie'], [min_x0, min_y0, max_x1, max_y1]))
    masks.append([min_x0, min_y0, max_x1, min_y0, max_x1, max_y1, min_x0, max_y1])
    return bounding_boxes, masks

In [14]:
def preprocess_figureqa(objects, orig_folder, split):
    main_dict = {}

    for image in tqdm(objects):
        image_num = image['image_index']
        
        
        
        main_dict[str(image_num)] = {}
        bounding_boxes = []
        masks = []
        # Get text elements x-axis labels, y-axis labels, leegnd labels. 
        # Legend
        if 'legend' in image['general_figure_info']:
            legend_bb = image['general_figure_info']['legend']['bbox']
            legend_x0, legend_y0, legend_w, legend_h = float(legend_bb["x"]), float(legend_bb["y"]), float(legend_bb["w"]), float(legend_bb["h"])
            legend_x1, legend_y1 = legend_x0 + legend_w, legend_y0 + legend_h 
            bounding_boxes.append((objects_classes['Legend'], [legend_x0, legend_y0, legend_x1, legend_y1]))
            masks.append([legend_x0, legend_y0, legend_x1, legend_y0, legend_x1, legend_y1, legend_x0, legend_y1])
            for item in image['general_figure_info']['legend']['items']:
                label_bbox = item['label']['bbox']
                label_x0, label_y0, label_h, label_w = float(label_bbox['x']), float(label_bbox['y']), float(label_bbox['h']), float(label_bbox['w'])
                label_x1, label_y1 = label_x0 + label_w, label_y0 + label_h
                bounding_boxes.append((objects_classes['LegendLabel'], [label_x0, label_y0, label_x1, label_y1]))
                masks.append([label_x0, label_y0, label_x1, label_y0, label_x1, label_y1, label_x0, label_y1])
                patch_bbox = item['preview']['bbox']
                patch_x0, patch_y0, patch_h, patch_w = float(patch_bbox['x']), float(patch_bbox['y']), float(patch_bbox['h']), float(patch_bbox['w'])
                patch_x1, patch_y1 = patch_x0 + patch_w, patch_y0 + patch_h
                bounding_boxes.append((objects_classes['LegendPreview'], [patch_x0, patch_y0, patch_x1, patch_y1]))
                masks.append([patch_x0, patch_y0, patch_x1, patch_y0, patch_x1, patch_y1, patch_x0, patch_y1])
        # Plot area
        if 'plot_info' in image['general_figure_info']:
            plot_area = image['general_figure_info']['plot_info']['bbox']
            plot_x0 = float(plot_area['x'])
            plot_y0 = float(plot_area['y'])
            plot_h = float(plot_area['h'])
            plot_w = float(plot_area['w'])
            plot_x1 = plot_x0 + plot_w
            plot_y1 = plot_y0 + plot_h
            bounding_boxes.append((objects_classes['PlotArea'], [plot_x0, plot_y0, plot_x1, plot_y1]))
            masks.append([plot_x0, plot_y0, plot_x1, plot_y0, plot_x1, plot_y1, plot_x0, plot_y1])
           
        # Title
        if 'title' in image['general_figure_info']:
            bbox = image['general_figure_info']['title']['bbox']
            x0, y0, h, w = float(bbox['x']), float(bbox['y']), float(bbox['h']), float(bbox['w'])
            x1 = x0 + w
            y1 = y0 + h
            bounding_boxes.append((objects_classes['ChartTitle'], [x0, y0, x1, y1]))
            masks.append([x0, y0, x1, y0, x1, y1, x0, y1])
        # X axis.
        if 'x_axis' in image['general_figure_info']:
            bbox = image['general_figure_info']['x_axis']['label']['bbox']
            x0, y0, h, w = float(bbox['x']), float(bbox['y']), float(bbox['h']), float(bbox['w'])
            x1 = x0 + w
            y1 = y0 + h
            bounding_boxes.append((objects_classes['xAxisTitle'], [x0, y0, x1, y1]))
            masks.append([x0, y0, x1, y0, x1, y1, x0, y1])
                
                
            labels_bboxes = image['general_figure_info']['x_axis']['major_labels']['bboxes']
            for bbox in labels_bboxes:
                x0, y0, h, w = float(bbox['x']), float(bbox['y']), float(bbox['h']), float(bbox['w'])
                x1 = x0 + w
                y1 = y0 + h
                bounding_boxes.append((objects_classes['xAxisLabel'], [x0, y0, x1, y1]))
                masks.append([x0, y0, x1, y0, x1, y1, x0, y1])

        # Y axis.
        if 'y_axis' in image['general_figure_info']:
            bbox = image['general_figure_info']['y_axis']['label']['bbox']
            x0, y0, h, w = float(bbox['x']), float(bbox['y']), float(bbox['h']), float(bbox['w'])
            x1 = x0 + w
            y1 = y0 + h
            bounding_boxes.append((objects_classes['yAxisTitle'], [x0, y0, x1, y1]))
            masks.append([x0, y0, x1, y0, x1, y1, x0, y1])
            
            labels_bboxes = image['general_figure_info']['y_axis']['major_labels']['bboxes']
            for bbox in labels_bboxes:
                x0, y0, h, w = float(bbox['x']), float(bbox['y']), float(bbox['h']), float(bbox['w'])
                x1 = x0 + w
                y1 = y0 + h
                bounding_boxes.append((objects_classes['yAxisLabel'], [x0, y0, x1, y1]))
                masks.append([x0, y0, x1, y0, x1, y1, x0, y1])

        # Get boxes and Masks for the bars, lines, and pie slices. 
        if image['type'] in ['line']:#, 'dot_line']:
            bounding_boxes_r, masks_r = preprocess_line_chart(image)
        elif image['type'] == 'dot_line':
            bounding_boxes_r, masks_r = preprocess_dot_line_chart(image)
        elif image['type'] in ['vbar_categorical', 'hbar_categorical']:
            bounding_boxes_r, masks_r = preprocess_bar_chart(image)
        elif image['type'] == 'pie':
            bounding_boxes_r, masks_r = preprocess_pie_chart(image)

        bounding_boxes.extend(bounding_boxes_r)
        masks.extend(masks_r)
        
        # Add all boxes and masks to the main dictionaries. 
        main_dict[str(image_num)]['bboxes'] = bounding_boxes
        main_dict[str(image_num)]['masks'] = masks
        
        # Draw the processed annotations to validate them. 
        #img = cv2.imread(orig_folder+split+"/png/"+str(image_num)+".png")
        #cv2.imwrite(save_folder+"images_"+split+"/Figureqa_"+str(image_num)+".png", img)
        #img = draw_bboxes(img, bounding_boxes)
        #img = draw_masks(img, masks)
        #plt.imshow(img)
        #plt.show()
    return main_dict

In [15]:
def draw_bboxes(img, boxes):
    for box in boxes:
        x0, y0, x1, y1 = box[1]
        img = cv2.rectangle(img, (int(x0), int(y0)), (int(x1), int(y1)), (255,0,0), 1)
    return img

In [16]:
def draw_masks(img, masks):
    reordered_masks = []
    for mask in masks:
        reordered_mask = []
        for i in range(0, len(mask), 2):
            x, y = mask[i], mask[i+1]
            reordered_mask.append([x, y])
        reordered_masks.append(reordered_mask)
    for mask in reordered_masks:
        pts = np.array(mask, np.int32)
        pts = pts.reshape((-1, 1, 2))
        img = cv2.polylines(img, [pts], True, (255, 0, 0), 2)
    return img

# Process The Charts

In [8]:
figureqa_folder = "D:/York University/ChartQA/FigureQA/"
split = "train1"

In [10]:
# Load Annotations
with open(figureqa_folder+split+"/annotations.json", 'rb')as f:
    annotations = json.load(f)

In [17]:
# Process Annotations
processed_annotations = preprocess_figureqa(annotations, figureqa_folder, split)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:23<00:00, 4297.84it/s]


In [18]:
with open("train_dict_123.pickle", 'wb') as handle:
    pickle.dump(processed_annotations, handle, protocol=pickle.HIGHEST_PROTOCOL)